In [7]:
# Brain Tumor Classification and Explainability Pipeline

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import shap


In [9]:

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [10]:

# Data Transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:

# Dataset and DataLoader
data_dir = "C:\Users\Papince Gupta\OneDrive\Desktop\semester 6\SE\project\Nidaan_AI\brain_tumor_mri_dataset"
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Training'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Testing'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [13]:
# Model Initialization
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 classes
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/home/tanuj/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tanuj/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/tanuj/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:12<00:00, 8.31MB/s]


In [14]:

# Training Pipeline
def train(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

    print('Training complete.')

In [15]:

# Explanation using LIME
def explain_with_lime(model, image, class_names):
    explainer = lime_image.LimeImageExplainer()
    
    def predict(input):
        input = torch.tensor(input.transpose((0, 3, 1, 2)), dtype=torch.float32).to(device)
        outputs = model(input)
        return torch.nn.functional.softmax(outputs, dim=1).detach().cpu().numpy()

    explanation = explainer.explain_instance(
        image.astype('double'), predict, top_labels=4, hide_color=0
    )

    image, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, hide_rest=False)
    plt.imshow(mark_boundaries(image, mask))
    plt.title(f'Explanation for {class_names[explanation.top_labels[0]]}')
    plt.show()

In [16]:

# Explanation using Grad-CAM
def explain_with_gradcam(model, image, target_layer, class_idx):
    cam = GradCAM(model=model, target_layers=[target_layer])
    grayscale_cam = cam(input_tensor=image.unsqueeze(0).to(device), targets=[ClassifierOutputTarget(class_idx)])[0]
    visualization = show_cam_on_image(image.permute(1, 2, 0).cpu().numpy(), grayscale_cam, use_rgb=True)
    plt.imshow(visualization)
    plt.title(f'Grad-CAM for class {class_idx}')
    plt.show()

# Explanation using SHAP
def explain_with_shap(model, image, background):
    explainer = shap.GradientExplainer(model, background)
    shap_values = explainer.shap_values(image.unsqueeze(0).to(device))
    shap.image_plot(shap_values, image.unsqueeze(0).cpu().numpy())

In [18]:

# Main Execution
if __name__ == "__main__":
    train(model, train_loader, val_loader, epochs=10)
    torch.save(model.state_dict(), "brain_tumor_model.pth")
    print("Model saved successfully.")

In [ ]:
# Brain Tumor Classification and Explainability Pipeline

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import pickle
from lime import lime_image
from skimage.segmentation import mark_boundaries
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import shap
from torch.optim import lr_scheduler

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dataset and DataLoader
data_dir = "C:\Users\Papince Gupta\OneDrive\Desktop\semester 6\SE\project\Nidaan_AI\brain_tumor_mri_dataset"
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Training'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'Testing'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Initialization
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 classes
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

# Training Pipeline
def train(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        val_loss = 0.0
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        scheduler.step(avg_val_loss)

    print('Training complete.')

# Explanation using LIME
def explain_with_lime(model, image, class_names):
    explainer = lime_image.LimeImageExplainer()
    
    def predict(input):
        input = torch.tensor(input.transpose((0, 3, 1, 2)), dtype=torch.float32).to(device)
        outputs = model(input)
        return torch.nn.functional.softmax(outputs, dim=1).detach().cpu().numpy()

    explanation = explainer.explain_instance(
        image.astype('double'), predict, top_labels=4, hide_color=0
    )

    image, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, hide_rest=False)
    plt.imshow(mark_boundaries(image, mask))
    plt.title(f'Explanation for {class_names[explanation.top_labels[0]]}')
    plt.show()

# Explanation using Grad-CAM
def explain_with_gradcam(model, image, target_layer, class_idx):
    cam = GradCAM(model=model, target_layers=[target_layer])
    grayscale_cam = cam(input_tensor=image.unsqueeze(0).to(device), targets=[ClassifierOutputTarget(class_idx)])[0]
    visualization = show_cam_on_image(image.permute(1, 2, 0).cpu().numpy(), grayscale_cam, use_rgb=True)
    plt.imshow(visualization)
    plt.title(f'Grad-CAM for class {class_idx}')
    plt.show()

# Explanation using SHAP
def explain_with_shap(model, image, background):
    explainer = shap.GradientExplainer(model, background)
    shap_values = explainer.shap_values(image.unsqueeze(0).to(device))
    shap.image_plot(shap_values, image.unsqueeze(0).cpu().numpy())




/home/tanuj/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tanuj/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
if __name__ == "__main__":
    train(model, train_loader, val_loader, epochs=10)
    torch.save(model.state_dict(), "brain_tumor_model.pth")
    with open("brain_tumor_model.pkl", "wb") as f:
        pickle.dump(model.state_dict(), f)
    print("Model saved successfully in both .pth and .pkl formats.")

Epoch [1/10], Train Loss: 0.3848, Val Loss: 1.0328
Epoch [2/10], Train Loss: 0.2016, Val Loss: 0.7852
Epoch [3/10], Train Loss: 0.1648, Val Loss: 9.2042
Epoch [4/10], Train Loss: 0.1768, Val Loss: 0.5896
Epoch [5/10], Train Loss: 0.0821, Val Loss: 0.4599
Epoch [6/10], Train Loss: 0.0900, Val Loss: 0.6548
Epoch [7/10], Train Loss: 0.0693, Val Loss: 0.7901
Epoch [8/10], Train Loss: 0.0647, Val Loss: 1.3631
Epoch [9/10], Train Loss: 0.0735, Val Loss: 1.2536
Epoch [10/10], Train Loss: 0.0199, Val Loss: 0.5158
Training complete.
Model saved successfully in both .pth and .pkl formats.
